## Celda 1: Definición de la Clase de Limpieza

In [1]:
## H2.ipynb - Celda 1: Definición de la Clase de Limpieza (COMPLETA)

import pandas as pd
import numpy as np
import re
from datetime import datetime
import unicodedata
import os
from typing import Dict, List, Any, Optional
# Importaciones para el reporte de calidad
import seaborn as sns
import matplotlib.pyplot as plt

# --- INICIO CLASE LimpiezaAutomatizada (Código COMPLETO) ---

class LimpiezaAutomatizada:
    """
    Sistema automatizado y reutilizable para limpieza de datos
    """
    
    def __init__(self, archivo_csv: str, config_limpieza: Dict = None):
        self.archivo_csv = archivo_csv
        self.df = None
        self.estadisticas_limpieza = {}
        self.config_limpieza = config_limpieza or self._configuracion_predeterminada()
        
    def _configuracion_predeterminada(self) -> Dict:
        """Configuración predeterminada para limpieza"""
        return {
            'mapeo_columnas': {
                'fecha': ['fecha', 'date', 'fecha_venta', 'timestamp'],
                'producto': ['producto', 'product', 'item', 'descripcion'],
                'tipo_producto': ['tipo_producto', 'categoria', 'category'],
                'cantidad': ['cantidad', 'qty', 'quantity', 'unidades'],
                'precio_unitario': ['precio_unitario', 'precio', 'price', 'unit_price'],
                'total_ventas': ['total_ventas', 'venta_total', 'total', 'amount', 'importe'],
                'tipo_venta': ['tipo_venta', 'canal_venta', 'channel'],
                'tipo_cliente': ['tipo_cliente', 'customer_type', 'segmento_cliente'],
                'descuento': ['descuento', 'discount'],
                'costo_envio': ['costo_envio', 'shipping_cost'],
                'ciudad': ['ciudad', 'city', 'localidad'],
                'pais': ['pais', 'country'],
                'region': ['region', 'state', 'estado']
            },
            'reglas_limpieza': {
                'texto': { 'case': 'title' }, 
                'numero': {
                    'min_value': 0, 
                    'max_value': 1000000, 
                    'decimales': 2
                },
                'fecha': {
                    'formatos': ['%Y-%m-%d', '%d/%m/%Y', '%m/%d/%Y', '%Y-%m-%d %H:%M:%S', 
                                 '%d-%m-%Y', '%m-%d-%Y', '%Y/%m/%d'],
                    'rango_min': '2020-01-01',
                    'rango_max': '2025-12-31'
                }
            },
            'columnas_orden_preferido': [
                'fecha', 'producto', 'tipo_producto', 'cantidad', 'precio_unitario',
                'ciudad', 'pais', 'tipo_venta', 'tipo_cliente', 'descuento', 'costo_envio', 'total_ventas'
            ]
        }
    
    # --- MÉTODOS DE UTILIDAD Y VALIDACIÓN (No cambiaron) ---
    def _normalizar_texto(self, texto: Any) -> str:
        if pd.isna(texto): return ""
        texto = str(texto).lower().strip()
        texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
        texto = re.sub(r'[^a-z0-9\s]', '', texto)
        return texto

    def _es_numerico(self, valor: str) -> bool:
        try:
            valor_limpio = re.sub(r'[^0-9.\-]', '', str(valor))
            if not valor_limpio or valor_limpio == '-': return False
            float(valor_limpio)
            return True
        except:
            return False

    def _es_fecha_valida(self, valor: str) -> bool:
        formatos = self.config_limpieza['reglas_limpieza']['fecha']['formatos']
        for formato in formatos:
            try:
                datetime.strptime(str(valor), formato)
                return True
            except ValueError:
                continue
        try:
            pd.to_datetime(valor, errors='raise')
            return True
        except:
            return False
            
    def _determinar_tipo_columna(self, serie: pd.Series) -> str:
        if serie.empty or serie.dropna().empty: return 'desconocido'
        serie_str = serie.dropna().astype(str)
        if len(serie_str) == 0: return 'desconocido'
        
        if serie_str.apply(self._es_fecha_valida).sum() / len(serie_str) > 0.5:
            return 'fecha'
        
        if serie_str.apply(self._es_numerico).sum() / len(serie_str) > 0.8:
            return 'numero'
        
        return 'texto'

    # --- MÉTODOS DE LIMPIEZA ESPECÍFICA (No cambiaron) ---
    # (Se omite el código de _limpiar_texto, _limpiar_numero, _limpiar_fecha, _limpiar_booleano, _limpiar_numero_robusto, _aplicar_limpieza_por_tipo para brevedad, pero debe estar completo en la celda)
    def _limpiar_texto(self, serie: pd.Series) -> pd.Series:
        def limpiar_valor(valor):
            if pd.isna(valor): return np.nan
            valor_str = str(valor).strip()
            valor_limpio = re.sub(r'[^a-zA-Z0-9áéíóúÁÉÍÓÚñÑüÜ\s\-_\.]', ' ', valor_str)
            valor_limpio = re.sub(r'\s+', ' ', valor_limpio).strip()
            if not valor_limpio: return np.nan
            
            config_case = self.config_limpieza['reglas_limpieza']['texto']['case']
            if config_case == 'lower': return valor_limpio.lower()
            elif config_case == 'upper': return valor_limpio.upper()
            elif config_case == 'title': return valor_limpio.title()
            else: return valor_limpio
        
        return serie.apply(limpiar_valor)
        
    def _limpiar_numero(self, serie: pd.Series) -> pd.Series:
        def limpiar_valor(valor):
            if pd.isna(valor): return np.nan
            try:
                valor_limpio = re.sub(r'[^0-9.\-]', '', str(valor))
                if not valor_limpio or valor_limpio == '-': return np.nan
                
                numero = float(valor_limpio)
                
                min_val = self.config_limpieza['reglas_limpieza']['numero']['min_value']
                max_val = self.config_limpieza['reglas_limpieza']['numero']['max_value']
                decimales = self.config_limpieza['reglas_limpieza']['numero']['decimales']
                
                if numero < min_val or numero > max_val: return np.nan
                
                return round(numero, decimales)
            except (ValueError, TypeError):
                return np.nan
        
        return serie.apply(limpiar_valor)

    def _limpiar_fecha(self, serie: pd.Series) -> pd.Series:
        def limpiar_valor(valor):
            if pd.isna(valor): return np.nan
            try:
                formatos = self.config_limpieza['reglas_limpieza']['fecha']['formatos']
                fecha = None
                for formato in formatos:
                    try:
                        fecha = datetime.strptime(str(valor), formato)
                        break
                    except ValueError:
                        continue
                if fecha is None: fecha = pd.to_datetime(valor, errors='coerce')
                if pd.isna(fecha): return np.nan
                rango_min = pd.to_datetime(self.config_limpieza['reglas_limpieza']['fecha']['rango_min'])
                rango_max = pd.to_datetime(self.config_limpieza['reglas_limpieza']['fecha']['rango_max'])
                if pd.to_datetime(fecha) < rango_min or pd.to_datetime(fecha) > rango_max:
                    return np.nan
                return fecha.strftime('%Y-%m-%d')
            except Exception as e:
                return np.nan
        return serie.apply(limpiar_valor)

    def _limpiar_booleano(self, serie: pd.Series) -> pd.Series:
        mapa_valores = {
            'true': True, 'false': False, '1': True, '0': False,
            'si': True, 'no': False, 'sí': True, 'yes': True, 
            'verdadero': True, 'falso': False, 't': True, 'f': False, 'v': True
        }
        def limpiar_valor(valor):
            if pd.isna(valor): return np.nan
            try:
                valor_str = str(valor).lower().strip()
                valor_str = re.sub(r'\s+', ' ', valor_str).strip()
                return mapa_valores.get(valor_str, np.nan)
            except:
                return np.nan
        return serie.apply(limpiar_valor)

    def _limpiar_numero_robusto(self, serie: pd.Series) -> pd.Series:
        serie_limpia = pd.to_numeric(serie, errors='coerce')
        if serie_limpia.isna().sum() > len(serie) * 0.5:
            print(f"      Muchos valores no numéricos en {serie.name}, aplicando limpieza manual...")
            serie_limpia = self._limpiar_numero(serie) 
        min_val = self.config_limpieza['reglas_limpieza']['numero']['min_value']
        decimales = self.config_limpieza['reglas_limpieza']['numero']['decimales']
        serie_limpia = serie_limpia.clip(lower=min_val)
        if decimales is not None:
            serie_limpia = serie_limpia.round(decimales)
        return serie_limpia

    def _aplicar_limpieza_por_tipo(self):
        print("🧹 Aplicando limpieza por tipo de dato...")
        columnas_numericas_criticas = ['cantidad', 'precio_unitario', 'descuento', 'costo_envio']
        for columna in columnas_numericas_criticas:
            if columna in self.df.columns:
                self.df[columna] = self._limpiar_numero_robusto(self.df[columna])
                if columna in ['cantidad', 'precio_unitario']: self.df[columna].fillna(0, inplace=True) 
        for columna in self.df.columns:
            if columna in columnas_numericas_criticas: continue
            if columna not in self.df.columns or self.df[columna].empty: continue
            tipo = self._determinar_tipo_columna(self.df[columna])
            
            try:
                if tipo == 'texto': self.df[columna] = self._limpiar_texto(self.df[columna])
                elif tipo == 'numero': self.df[columna] = self._limpiar_numero_robusto(self.df[columna])
                elif tipo == 'fecha': self.df[columna] = self._limpiar_fecha(self.df[columna])
                elif tipo == 'booleano': self.df[columna] = self._limpiar_booleano(self.df[columna])
                else: self.df[columna] = self._limpiar_texto(self.df[columna])
            except Exception as e:
                print(f"   ⚠ Advertencia limpiando {columna}: {e}")
        print("   ✅ Limpieza por tipo de dato completada.")
    # --- FIN MÉTODOS DE LIMPIEZA ESPECÍFICA ---


    # --- MÉTODOS DE CÁLCULO Y ESTRUCTURA (No cambiaron) ---
    def _calcular_total_ventas(self):
        if 'total_ventas' not in self.df.columns or self.df['total_ventas'].isnull().all():
            if 'cantidad' in self.df.columns and 'precio_unitario' in self.df.columns:
                print("💰 Calculando columna total_ventas...")
                cantidad = self.df['cantidad'].fillna(0)
                precio = self.df['precio_unitario'].fillna(0)
                self.df['total_ventas'] = cantidad * precio
                if 'descuento' in self.df.columns:
                    descuento = self.df['descuento'].fillna(0)
                    self.df['total_ventas'] = self.df['total_ventas'] * (1 - descuento / 100)
                print(f"   ✅ Total_ventas calculado.")
            else:
                print("⚠ No se puede calcular total_ventas - faltan columnas requeridas (cantidad/precio_unitario)")
    
    def _reorganizar_datos_mal_estructurados(self, df: pd.DataFrame) -> pd.DataFrame:
        print("🔄 Reorganizando datos mal estructurados...")
        columnas_actuales = df.columns.tolist()
        if len(columnas_actuales) >= 10 and 'ciudad' not in df.columns:
            try:
                nuevos_datos = []
                for _, fila in df.iterrows():
                    nueva_fila = {
                        'ciudad': str(fila.iloc[0]) if len(fila) > 0 else '', 'fecha': str(fila.iloc[1]) if len(fila) > 1 else '',
                        'producto': str(fila.iloc[2]) if len(fila) > 2 else '', 'tipo_producto': str(fila.iloc[3]) if len(fila) > 3 else '',
                        'cantidad': str(fila.iloc[4]) if len(fila) > 4 else '', 'precio_unitario': str(fila.iloc[5]) if len(fila) > 5 else '',
                        'tipo_venta': str(fila.iloc[6]) if len(fila) > 6 else '', 'tipo_cliente': str(fila.iloc[7]) if len(fila) > 7 else '',
                        'descuento': str(fila.iloc[8]) if len(fila) > 8 else '', 'costo_envio': str(fila.iloc[9]) if len(fila) > 9 else '',
                        'pais': '' 
                    }
                    nuevos_datos.append(nueva_fila)
                df_corregido = pd.DataFrame(nuevos_datos)
                print(f"   ✅ Datos reorganizados: {len(df_corregido)} filas")
                return df_corregido
            except Exception as e:
                print(f"   ⚠ Error reorganizando datos: {e}")
                return df
        else:
            print("   ℹ️  Estructura de columnas parece correcta")
            return df
    
    def _mapear_columnas_automatico(self, columnas_originales: List[str]) -> Dict:
        mapeo = {}; columnas_mapeadas = set(); columnas_no_mapeadas = columnas_originales.copy()
        for nombre_estandar, variantes in self.config_limpieza['mapeo_columnas'].items():
            for columna_original in columnas_originales:
                if columna_original in columnas_mapeadas: continue
                col_clean = self._normalizar_texto(columna_original)
                for variante in variantes:
                    variante_clean = self._normalizar_texto(variante)
                    if variante_clean == col_clean or (variante_clean in col_clean or col_clean in variante_clean) and len(col_clean) > 2:
                        if nombre_estandar not in mapeo.values():
                            mapeo[columna_original] = nombre_estandar
                            columnas_mapeadas.add(columna_original)
                            if columna_original in columnas_no_mapeadas: columnas_no_mapeadas.remove(columna_original)
                            break
        nombres_estandar_usados = set(mapeo.values())
        temp_no_mapeadas = columnas_no_mapeadas.copy()
        for i, columna in enumerate(temp_no_mapeadas):
            if columna not in mapeo:
                nombre_estandar = f"columna_{i+1}"
                while nombre_estandar in nombres_estandar_usados:
                    i += 1
                    nombre_estandar = f"columna_{i+1}"
                mapeo[columna] = nombre_estandar
                nombres_estandar_usados.add(nombre_estandar)
        return mapeo

    def _detectar_y_corregir_pais(self, df: pd.DataFrame) -> pd.DataFrame:
        print("   🗺️  Detectando países basado en ciudades...")
        mapeo_ciudad_pais = {
            'bogota': 'Colombia', 'medellin': 'Colombia', 'cali': 'Colombia',
            'new york': 'Estados Unidos', 'madrid': 'España', 'ciudad de mexico': 'México', 
            'buenos aires': 'Argentina', 'sao paulo': 'Brasil', 'lima': 'Perú', 'santiago': 'Chile' 
        }
        def detectar_pais(ciudad):
            if pd.isna(ciudad) or str(ciudad).strip() == '': return 'Desconocido'
            ciudad_clean = self._normalizar_texto(ciudad)
            for ciudad_mapeo, pais in mapeo_ciudad_pais.items():
                if ciudad_clean == self._normalizar_texto(ciudad_mapeo): return pais
            return 'Desconocido'
        if 'ciudad' in df.columns:
            df['pais'] = df['ciudad'].apply(detectar_pais)
            print(f"   ✅ Países detectados: {df['pais'].value_counts().to_dict()}")
        return df

    def _eliminar_columnas_duplicadas(self, df: pd.DataFrame) -> pd.DataFrame:
        print("🔍 Buscando columnas duplicadas...")
        columnas_a_mantener = []; columnas_vistas = set()
        for columna in df.columns:
            if columna not in columnas_vistas:
                columnas_a_mantener.append(columna)
                columnas_vistas.add(columna)
            else:
                print(f"   🗑️  Eliminando columna duplicada: '{columna}'")
        return df[columnas_a_mantener]
    
    def _reordenar_columnas(self, df: pd.DataFrame) -> pd.DataFrame:
        orden_preferido = self.config_limpieza.get('columnas_orden_preferido', [])
        columnas_ordenadas = [col for col in orden_preferido if col in df.columns]
        columnas_restantes = [col for col in df.columns if col not in columnas_ordenadas]
        columnas_finales = columnas_ordenadas + sorted(columnas_restantes)
        if columnas_finales != df.columns.tolist():
            print("🔄 Reordenando columnas...")
            df = df[columnas_finales]
        return df
    # --- FIN MÉTODOS DE CÁLCULO Y ESTRUCTURA ---


    # --- MÉTODOS DE ESTADÍSTICAS Y MAIN ---
    def _calcular_estadisticas_limpieza(self, registros_originales: int):
        self.estadisticas_limpieza = {
            'registros_originales': registros_originales,
            'registros_finales': len(self.df),
            'columnas_finales': len(self.df.columns),
            'nulos_por_columna': self.df.isnull().sum().to_dict(),
            'registros_eliminados': registros_originales - len(self.df),
            'porcentaje_completitud': (1 - self.df.isnull().sum().sum() / (len(self.df) * len(self.df.columns))) * 100
        }
    
    def _mostrar_resumen_limpieza(self):
        print("\n" + "="*60)
        print("📊 RESUMEN DE LIMPIEZA AUTOMATIZADA")
        print("="*60)
        stats = self.estadisticas_limpieza
        print(f"📈 Registros originales: {stats['registros_originales']:,}")
        print(f"📈 Registros finales: {stats['registros_finales']:,}")
        print(f"📊 Columnas finales: {stats['columnas_finales']}")
        print(f"🗑️  Registros eliminados: {stats['registros_eliminados']:,}")
        print(f"✅ Completitud: {stats['porcentaje_completitud']:.1f}%")
        
        print("\n📋 COLUMNAS FINALES (ORDENADAS):")
        for i, columna in enumerate(self.df.columns, 1):
            nulos = stats['nulos_por_columna'][columna]
            total = len(self.df)
            porcentaje_valido = ((total - nulos)/total)*100
            print(f"   {i:2d}. {columna}: {total - nulos}/{total} válidos ({porcentaje_valido:.1f}%)")
        print("="*60)

    def guardar_datos_limpios(self, archivo_salida: str = "datos_limpios.csv"):
        """Guardar datos limpios"""
        try:
            self.df = self._eliminar_columnas_duplicadas(self.df)
            self.df.to_csv(archivo_salida, index=False, encoding='utf-8')
            print(f"\n💾 Datos guardados en: {archivo_salida}")
            return True
        except Exception as e:
            print(f"❌ Error guardando archivo: {e}")
            return False

    def generar_reporte_calidad(self, archivo_reporte: str = "reporte_calidad.png"):
        """Generar reporte de calidad de datos con visualizaciones (MÉTODO FALTANTE)"""
        try:
            fig, axes = plt.subplots(1, 2, figsize=(16, 6))
            
            # Gráfico de nulos por columna
            nulos_por_columna = self.df.isnull().sum().sort_values(ascending=False)
            sns.barplot(x=nulos_por_columna.values, y=nulos_por_columna.index, ax=axes[0], palette="viridis")
            axes[0].set_title('Valores Nulos por Columna')
            axes[0].set_xlabel('Cantidad de Valores Nulos')
            axes[0].set_ylabel('Columna')
            
            # Gráfico de tipos de datos
            tipos_datos = self.df.dtypes.astype(str).value_counts()
            axes[1].pie(tipos_datos.values, labels=tipos_datos.index, autopct='%1.1f%%', startangle=90, colors=sns.color_palette("pastel"))
            axes[1].set_title('Distribución de Tipos de Datos Finales')
            axes[1].axis('equal')
            
            plt.tight_layout()
            plt.savefig(archivo_reporte, dpi=300)
            plt.close(fig)
            print(f"📊 Reporte de calidad generado: {archivo_reporte}")
            return True
        except Exception as e:
            print(f"⚠ No se pudo generar reporte visual: {e}")
            return False

    def detectar_estructura(self) -> Dict:
        """Detectar automáticamente la estructura del archivo (MÉTODO FALTANTE)"""
        try:
            self.df = pd.read_csv(self.archivo_csv, nrows=1000)
            self.df = self._reorganizar_datos_mal_estructurados(self.df)
            mapeo_automatico = self._mapear_columnas_automatico(self.df.columns.tolist())
            
            df_temp = self.df.rename(columns=mapeo_automatico)
            tipos_datos = {}
            for col in df_temp.columns:
                tipos_datos[col] = {'tipo_probable': self._determinar_tipo_columna(df_temp[col]), 'ejemplos': df_temp[col].dropna().unique().tolist()[:3]}

            return {
                'mapeo_propuesto': mapeo_automatico,
                'tipos_datos': tipos_datos,
                'muestra_datos': df_temp.head(3).to_dict('records')
            }
        except Exception as e:
            print(f"❌ Error detectando estructura: {e}")
            return {}

    def aplicar_limpieza(self, mapeo_personalizado: Dict = None) -> pd.DataFrame:
        """Aplicar limpieza completa a los datos (MAIN METHOD)"""
        print("\n🧹 APLICANDO LIMPIEZA AUTOMATIZADA...")
        
        try:
            self.df = pd.read_csv(self.archivo_csv)
            registros_originales = len(self.df)
            
            # Flujo de Limpieza
            self.df = self._reorganizar_datos_mal_estructurados(self.df)
            mapeo_final = mapeo_personalizado or self._mapear_columnas_automatico(self.df.columns.tolist())
            self.df = self.df.rename(columns=mapeo_final)
            print("✅ Columnas renombradas")
            self.df = self._eliminar_columnas_duplicadas(self.df)
            self.df = self._detectar_y_corregir_pais(self.df)
            self._aplicar_limpieza_por_tipo()
            self._calcular_total_ventas()
            self.df = self._reordenar_columnas(self.df)
            
            # Finalización
            self._calcular_estadisticas_limpieza(registros_originales)
            self._mostrar_resumen_limpieza()
            
            return self.df
            
        except Exception as e:
            print(f"❌ Error en limpieza: {e}")
            import traceback
            traceback.print_exc()
            return pd.DataFrame()

# --- FIN CLASE LimpiezaAutomatizada ---


# --- FUNCIÓN DE USO RÁPIDO (FALTANTE) ---

def limpiar_csv_automatico(archivo_csv: str, archivo_salida: str = "datos_limpios.csv") -> pd.DataFrame:
    """
    Función de uso rápido para limpieza automática
    """
    limpiador = LimpiezaAutomatizada(archivo_csv)
    
    # 1. Detectar estructura
    estructura = limpiador.detectar_estructura()
    print("\n🔍 MAPEO AUTOMÁTICO PROPUESTO:")
    for orig, nuevo in estructura['mapeo_propuesto'].items():
        print(f"   '{orig}' → '{nuevo}'")
    
    # 2. Aplicar limpieza
    df_limpio = limpiador.aplicar_limpieza()
    
    if not df_limpio.empty:
        # 3. Persistir y reportar
        limpiador.guardar_datos_limpios(archivo_salida)
        limpiador.generar_reporte_calidad()
    
    return df_limpio

if __name__ == "__main__":
    # Esta sección no se ejecuta en el notebook, pero es buena práctica mantenerla.
    pass

## Celda 2: Ejecución del Flujo ETL (E/T)

Esta celda ejecuta la limpieza y genera las tablas de dimensión y la tabla de hechos con sus claves foráneas.

In [2]:
## H2.ipynb - Celda 2: Ejecución de Limpieza y Generación del Esquema Estrella

# 1. Definir el archivo de entrada
ARCHIVO_VENTAS = 'ventas.csv' # Asegúrate de que este archivo exista

# 2. Inicializar y ejecutar la limpieza usando la función de uso rápido
print("--- 1. Limpieza y Normalización de Hechos (Fact Table) ---")
df_hechos_limpio = limpiar_csv_automatico(ARCHIVO_VENTAS, archivo_salida='ventas_temp_limpio.csv')

# --- 3. GENERACIÓN DE TABLAS DE DIMENSIÓN ---

if not df_hechos_limpio.empty:
    
    print("\n--- 2. Extracción y Generación de Dimensiones ---")
    
    # a) Dimensión: TIPOS_CLIENTES (Dim_Tipo_Cliente)
    df_tipos_clientes = df_hechos_limpio[['tipo_cliente']].drop_duplicates().dropna().reset_index(drop=True)
    df_tipos_clientes['tipo_cliente_id'] = df_tipos_clientes.index + 1 
    DF_DIM_TIPOS_CLIENTES = df_tipos_clientes[['tipo_cliente_id', 'tipo_cliente']].rename(
        columns={'tipo_cliente': 'nombre_tipo_cliente'}
    ).copy()
    print("✅ Generada Dimensión TIPOS_CLIENTES")


    # b) Dimensión: PRODUCTOS (Dim_Producto)
    df_productos = df_hechos_limpio[['producto', 'tipo_producto']].drop_duplicates().dropna().reset_index(drop=True)
    df_productos['producto_id'] = df_productos.index + 1 
    DF_DIM_PRODUCTOS = df_productos[['producto_id', 'producto', 'tipo_producto']].rename(
        columns={'producto': 'nombre_producto', 'tipo_producto': 'nombre_tipo_producto'}
    ).copy()
    print("✅ Generada Dimensión PRODUCTOS")

    
    # --- 4. Mapeo de Claves Foráneas (FK) en la Tabla de Hechos ---
    print("\n--- 3. Mapeo de Claves Foráneas (FK) en la Tabla de Hechos ---")
    
    # Mapeo de Tipo Cliente
    map_tipo_cliente = DF_DIM_TIPOS_CLIENTES.set_index('nombre_tipo_cliente')['tipo_cliente_id'].to_dict()
    df_hechos_limpio['tipo_cliente_fk'] = df_hechos_limpio['tipo_cliente'].map(map_tipo_cliente).fillna(0).astype(int) # Usar 0 para Desconocido/Nulo
    
    # Mapeo de Producto
    map_producto = DF_DIM_PRODUCTOS.set_index('nombre_producto')['producto_id'].to_dict()
    df_hechos_limpio['producto_fk'] = df_hechos_limpio['producto'].map(map_producto).fillna(0).astype(int) # Usar 0 para Desconocido/Nulo
    
    
    # --- 5. TABLA DE HECHOS FINAL (Fact_Ventas) ---
    columnas_fact_table = [
        'producto_fk', 'tipo_cliente_fk', # <--- Claves Foráneas
        'fecha', 'ciudad', 'tipo_venta', 'pais', # <--- Dimensiones Degradadas
        'cantidad', 'precio_unitario', 'descuento', 'costo_envio', 'total_ventas' # <--- Métricas
    ]
    
    DF_FACT_VENTAS = df_hechos_limpio[columnas_fact_table].copy()
    
    print("\n--- Vista Previa del DataFrame de Hechos con FKs (Listo para Carga) ---")
    print(DF_FACT_VENTAS.head())
    print("-" * 50)
else:
    print("❌ El DataFrame de hechos no pudo ser limpiado/cargado.")

--- 1. Limpieza y Normalización de Hechos (Fact Table) ---
🔄 Reorganizando datos mal estructurados...
   ✅ Datos reorganizados: 1000 filas

🔍 MAPEO AUTOMÁTICO PROPUESTO:
   'fecha' → 'fecha'
   'producto' → 'producto'
   'tipo_producto' → 'tipo_producto'
   'cantidad' → 'cantidad'
   'precio_unitario' → 'precio_unitario'
   'tipo_venta' → 'tipo_venta'
   'tipo_cliente' → 'tipo_cliente'
   'descuento' → 'descuento'
   'costo_envio' → 'costo_envio'
   'ciudad' → 'ciudad'
   'pais' → 'pais'

🧹 APLICANDO LIMPIEZA AUTOMATIZADA...
🔄 Reorganizando datos mal estructurados...
   ✅ Datos reorganizados: 1250000 filas
✅ Columnas renombradas
🔍 Buscando columnas duplicadas...
   🗺️  Detectando países basado en ciudades...
   ✅ Países detectados: {'Desconocido': 923690, 'Colombia': 76580, 'Perú': 44859, 'Argentina': 44687, 'España': 44561, 'Chile': 44494, 'México': 35624, 'Estados Unidos': 35505}
🧹 Aplicando limpieza por tipo de dato...


C:\Users\ASUS\AppData\Local\Temp\ipykernel_4628\307013661.py:205: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  if columna in ['cantidad', 'precio_unitario']: self.df[columna].fillna(0, inplace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_4628\307013661.py:205: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting valu

   ✅ Limpieza por tipo de dato completada.
💰 Calculando columna total_ventas...
   ✅ Total_ventas calculado.
🔄 Reordenando columnas...

📊 RESUMEN DE LIMPIEZA AUTOMATIZADA
📈 Registros originales: 1,250,000
📈 Registros finales: 1,250,000
📊 Columnas finales: 12
🗑️  Registros eliminados: 0
✅ Completitud: 100.0%

📋 COLUMNAS FINALES (ORDENADAS):
    1. fecha: 1248620/1250000 válidos (99.9%)
    2. producto: 1250000/1250000 válidos (100.0%)
    3. tipo_producto: 1250000/1250000 válidos (100.0%)
    4. cantidad: 1250000/1250000 válidos (100.0%)
    5. precio_unitario: 1250000/1250000 válidos (100.0%)
    6. ciudad: 1250000/1250000 válidos (100.0%)
    7. pais: 1250000/1250000 válidos (100.0%)
    8. tipo_venta: 1250000/1250000 válidos (100.0%)
    9. tipo_cliente: 1250000/1250000 válidos (100.0%)
   10. descuento: 1248049/1250000 válidos (99.8%)
   11. costo_envio: 1248091/1250000 válidos (99.8%)
   12. total_ventas: 1250000/1250000 válidos (100.0%)
🔍 Buscando columnas duplicadas...

💾 Datos g

C:\Users\ASUS\AppData\Local\Temp\ipykernel_4628\307013661.py:379: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=nulos_por_columna.values, y=nulos_por_columna.index, ax=axes[0], palette="viridis")


📊 Reporte de calidad generado: reporte_calidad.png

--- 2. Extracción y Generación de Dimensiones ---
✅ Generada Dimensión TIPOS_CLIENTES
✅ Generada Dimensión PRODUCTOS

--- 3. Mapeo de Claves Foráneas (FK) en la Tabla de Hechos ---

--- Vista Previa del DataFrame de Hechos con FKs (Listo para Carga) ---
   producto_fk  tipo_cliente_fk       fecha        ciudad     tipo_venta  \
0           84                1  2025-10-30      Santiago         Online   
1           84                2  2025-11-17       Córdoba   Distribuidor   
2           89                2  2025-10-22  Barranquilla   Distribuidor   
3           75                2  2025-10-20      New York  Tienda_Física   
4           89                3  2025-10-20        Madrid   Distribuidor   

             pais  cantidad  precio_unitario  descuento  costo_envio  \
0           Chile       2.0           3681.0       0.20          0.0   
1     Desconocido       7.0           2321.0       0.15          0.0   
2     Desconocido    

## Celda 3: Persistencia de Datos Limpios

Esta celda guarda las tablas generadas en el disco, listas para la Carga (L) final en el DWH.

In [4]:
## H2.ipynb - Celda 3: Persistencia de Datos Limpios (Listo para Cargar en BD)

if 'DF_DIM_TIPOS_CLIENTES' in locals() and 'DF_FACT_VENTAS' in locals():
    
    # 1. Guardar la Dimensión TIPOS_CLIENTES
    RUTA_DIM_TIPOS_CLIENTES = 'carga_d_tipos_clientes.csv'
    DF_DIM_TIPOS_CLIENTES.to_csv(RUTA_DIM_TIPOS_CLIENTES, index=False, encoding='utf-8')
    print(f"✅ Dimensión TIPOS_CLIENTES guardada en: {RUTA_DIM_TIPOS_CLIENTES}")
    
    # 2. Guardar la Dimensión PRODUCTOS
    RUTA_DIM_PRODUCTOS = 'carga_d_productos.csv'
    DF_DIM_PRODUCTOS.to_csv(RUTA_DIM_PRODUCTOS, index=False, encoding='utf-8')
    print(f"✅ Dimensión PRODUCTOS guardada en: {RUTA_DIM_PRODUCTOS}")
    
    # 3. Guardar la Tabla de Hechos VENTAS
    RUTA_FACT_VENTAS = 'carga_f_ventas.csv'
    DF_FACT_VENTAS.to_csv(RUTA_FACT_VENTAS, index=False, encoding='utf-8')
    print(f"✅ Hechos VENTAS guardados en: {RUTA_FACT_VENTAS}")
    
    print("\n🎉 Proceso ETL (E/T) completado. Los CSVs están listos para la Carga (L) en el Star Schema.")
else:
    print("❌ No se pudieron generar los DataFrames finales. Verifique la Celda 2.")

✅ Dimensión TIPOS_CLIENTES guardada en: carga_d_tipos_clientes.csv
✅ Dimensión PRODUCTOS guardada en: carga_d_productos.csv
✅ Hechos VENTAS guardados en: carga_f_ventas.csv

🎉 Proceso ETL (E/T) completado. Los CSVs están listos para la Carga (L) en el Star Schema.
